In [2]:
# из колаба команды вытаскиваю все возможные импорты чтобы дальше не думать об этом
import urllib, requests, socket, re, lxml, io, bs4, sqlite3, pandas, sqlalchemy
from requests.compat import urljoin, quote_plus, urlparse, unquote
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
import re
import chardet

**Маркетинговая стратегия. Поиск суперзвезды.**

Одна из основных стратегий продвижения нашей команды – коллабы со знаменитыми волейболистами и волейбольными командами. В этом файле – процесс обработки информации об этом: таблица с показателями спортсменов и ее дальнейшая визуализация (она пока в процессе :) но тоже скоро здесь появится). Также я подумала, что так как мы работаем с командами, + фанаты часто болеют за конкретные команды, надо сделать анализ того, с какими командами можно коллабиться. Для этого я для каждого спортсмена из таблицы захожу на его страницу и вытаскиваю инфу о клубе, в котором он играет (гистограмма для этой штуки тоже скоро появится). При разработке кода если долго не могла найти ошибку, консультировалась с чатомГПТ, что могло пойти не так, или просила совета у сокомандников.

In [11]:
url = 'https://volley.ru/calendar/01J1VV7JAR7KZS011D2BDE00D5/statistics'
response = requests.get(url)
encoding = chardet.detect(response.content)['encoding']
html = response.content.decode(encoding)
soup = BeautifulSoup(html, 'lxml')

# достаем имена
html_names = soup.find_all('td', {'data-field': 'name'})
names = []
for i in html_names:
  new_soup = BeautifulSoup(str(i), 'lxml')
  s = new_soup.find_all('a')
  names.append(s[0].text)

# результаты подач
html_serves = soup.find_all('td', {'data-field': 'total_serve_point'})
serves = []
for i in html_serves[1:]:
  serves.append(i.text)

# процент отличных приемов
html_digs = soup.find_all('td', {'class': 'stat-table-5', 'data-field': 'total_receive_excellent'})
digs = []
for i in html_digs[1:]:
  new_soup = BeautifulSoup(str(i), 'lxml')
  s = new_soup.find_all('span', {'class': 'ts-table__active-cell'})
  if s != []:
    s = s[0].text
    if '%' in s:
      s = s[:len(i) - 1]
    if '+' in s:
      s = s[1:]
    digs.append(s)
  else:
    digs.append('0')

# процент эффективных атак
html_hits = soup.find_all('td', {'class': 'stat-table-6', 'data-field': 'total_attack_point_percent'})
hits = []
for i in html_hits[1:]:
  new_soup = BeautifulSoup(str(i), 'lxml')
  s = new_soup.find_all('span', {'class': 'ts-table__active-cell'})
  if s != []:
    s = s[0].text
    if '%' in s:
      s = s[:len(i) - 1]
    if '+' in s:
      s = s[1:]
    hits.append(s)
  else:
    hits.append('0')

# очки на блоке
html_blocks = soup.find_all('td', {'class': 'stat-table-7', 'data-field': 'total_block'})
blocks = []
for i in html_blocks[1:]:
  new_soup = BeautifulSoup(str(i), 'lxml')
  s = new_soup.find_all('span', {'class': 'ts-table__active-cell'})
  if s != []:
    s = s[0].text
    if '%' in s:
      s = s[:len(i) - 1]
    if '+' in s:
      s = s[1:]
    blocks.append(s)
  else:
    blocks.append('0')
print(names)
print(serves)
print(digs)
print(hits)
print(blocks)

['Бабкевич Владислав', 'Железняков Павел', 'Мурашко Роман', 'Сурмачевский Андрей', 'Сиденко Егор', 'Дмитриев Семен', 'Папазов Виталий', 'Тетюхин Павел', 'Капранов Василий', 'Пакшин Роман', 'Мергарехо Осниель', 'Хлякин Михаил', 'Янт Марлон', 'Зубов Николай', 'Аль Хачдади Мохамед', 'Волков Дмитрий', 'Авдоченко Павел', 'Сапожков Максим', 'Поталюк Роман', 'Соколов Цветан', 'Рыбаков Евгений', 'Дикарев Виталий', 'Морозов Никита', 'Маркин Александр', 'Фиэль Родригес Давид', 'Казаченков Илья', 'Воронков Федор', 'Лызик Дмитрий', 'Родичев Алексей', 'Бочаров Максим', 'Ивович Марко', 'Лабинский Михаил', 'Клец Кирилл', 'Богдан Денис', 'Чжан Цзинъинь', 'Заболотников Георгий', 'Сподобец Илья', 'Курбанов Омар', 'Урсов Кирилл', 'Захватенков Александр', 'Крсманович Петар', 'Клюка Егор', 'Пискарев Иван', 'Шахбанмирзаев Ражабдибир', 'Эннс Корней', 'Пипуныров Матвей', 'Масько Артем', 'Подлесных Ярослав', 'Максимов Антон', 'Шкулявичус Романас', 'Пятыркин Александр', 'Бирюков Денис', 'Жук Дмитрий', 'Никитин 

Поиск крутых клубов по топ-игрокам (мужчины):

*И здесь, и для женщин, я беру выборку из топ-60 игроков, чтобы смотреть клубы по ним (изначально в датасете мужчин 258, женщин – 232).*

In [13]:
url = 'https://volley.ru/calendar/01J1VV7JAR7KZS011D2BDE00D5/statistics'
response = requests.get(url)
encoding = chardet.detect(response.content)['encoding']
html = response.content.decode(encoding)
soup = BeautifulSoup(html, 'lxml')

tables = soup.find_all('table')
ssylki = []

hrefs = tables[0].find_all('a')
for hre in hrefs:
 hre = str(hre) # вытащили список ссылок на страницы игроков
 index = hre.find('>')
 #print('https://volley.ru' + str(hre[9:44])) # сделали их корректными ссылками
 ssylki.append('https://volley.ru' + str(hre[9:44]))
ssylki # получили в адекватном виде список ссылкок

['https://volley.ru/players/01FQH659A2PK7V0G7YHJPT1YBC',
 'https://volley.ru/players/01GWJ7ZMYRQM2Q0ZYKW2D31XMM',
 'https://volley.ru/players/01GGF1PDDKN6T00C0FQCWP0XGT',
 'https://volley.ru/players/01GGEWNTA0VGWC0V7RP4RJ0N00',
 'https://volley.ru/players/01GWJ89PS8KYYM1754Z3YZ1AFK',
 'https://volley.ru/players/01GGCHA2NARQR70SF6NRYH001X',
 'https://volley.ru/players/01GGF39QXSE3R01PNFN7YM0HFF',
 'https://volley.ru/players/01GGF1P8C91MAT0XTVECGS1YB8',
 'https://volley.ru/players/01GDFYY2HZYVA11EYCBV3A0NJA',
 'https://volley.ru/players/01G79HKA0SXWBZ0QEY9NFF0G9R',
 'https://volley.ru/players/01J5N83Q6DHKK00RM0QD3N1E4F',
 'https://volley.ru/players/01GGF0FDYZ6GY2193ZQ0WH0S4F',
 'https://volley.ru/players/01J5RF7GKKKY350CT277Y21MP7',
 'https://volley.ru/players/01GGEQ790MDQ0H12HMKNJ00QJ9',
 'https://volley.ru/players/01H8BFG11A3H0G0EJCZBF01DYB',
 'https://volley.ru/players/01GGF03M3DD1B618EGJGWW1GR2',
 'https://volley.ru/players/01H8BFG1199F1R0THFHRKV0JE8',
 'https://volley.ru/players/01H

In [18]:
clubs = [] # в этой ячейке мы сделали список клубов сооответственно игрокам

for i in range(60):
  ssylka = ssylki[i]
  response = requests.get(ssylka)
  encoding = chardet.detect(response.content)['encoding']
  html = response.content.decode(encoding)
  soup = BeautifulSoup(html, 'lxml')
  html_urls = soup.find_all('a', {'class': 'itext-40'})
  # print(html_urls[0].text)
  clubs.append(html_urls[0].text)

In [21]:
# хочу список клубов словарем, где ключ -- название, а значение -- количество повторений: дальше мы с этой инфой будем строить графики
clubs_dict = {}
for item in clubs:
    if item in clubs_dict:
        clubs_dict[item] += 1
    else:
        clubs_dict[item] = 1
print(clubs_dict)

{' Зенит, Санкт-Петербург': 3, ' АСК, Нижний Новгород': 2, ' Енисей, Красноярск': 4, ' Динамо-Урал, Уфа': 5, ' Кузбасс, Кемерово': 4, ' Газпром-Югра, Сургут': 3, '  Белогорье, Белгород': 4, ' Факел Ямал, Новый Уренгой': 5, '  Динамо-ЛО, Ленинградская обл.': 3, ' Оренбуржье, Оренбург': 2, ' МГТУ, Москва': 2, '  Зенит-Казань, Казань': 3, ' Динамо, Москва': 4, '  Нова, Новокуйбышевск': 1, ' Локомотив, Новосибирск': 5}


Визуализация рейтинга клубов:

In [ ]:
# хочу построить график основных показателей, чтобы проанализировать, с кем лучше коллабиться, и составить гипотезы о том, кто может быть наиболее популярным

**Аналогично для женщин:**

In [ ]:
url = 'https://volley.ru/calendar/01J1YDG9TCC2MJ1RN5WM6H04C6/statistics'
response = requests.get(url)
encoding = chardet.detect(response.content)['encoding']  # определяем кодировку
html = response.content.decode(encoding)  # декодируем с норм кодировкой
soup = BeautifulSoup(html, 'lxml')

# достаем имена
html_wnames = soup.find_all('td', {'data-field': 'name'})

wnames = []
for i in html_wnames:
  new_soup = BeautifulSoup(str(i), 'lxml')
  s = new_soup.find_all('a')
  wnames.append(s[0].text)

# результаты подач
html_wserves = soup.find_all('td', {'data-field': 'total_serve_point'})

wserves = []
for i in html_wserves[1:]:
  wserves.append(i.text)

# процент отличных приемов
html_wdigs = soup.find_all('td', {'class': 'stat-table-5', 'data-field': 'total_receive_excellent'})
wdigs = []
for i in html_wdigs[1:]:
  new_soup = BeautifulSoup(str(i), 'lxml')
  s = new_soup.find_all('span', {'class': 'ts-table__active-cell'})
  if s != []:
    s = s[0].text
    if '%' in s:
      s = s[:len(i) - 1]
    if '+' in s:
      s = s[1:]
    wdigs.append(s)
  else:
    wdigs.append('0')

# процент эффективных атак
html_whits = soup.find_all('td', {'class': 'stat-table-6', 'data-field': 'total_attack_point_percent'})
whits = []
for i in html_whits[1:]:
  new_soup = BeautifulSoup(str(i), 'lxml')
  s = new_soup.find_all('span', {'class': 'ts-table__active-cell'})
  if s != []:
    s = s[0].text
    if '%' in s:
      s = s[:len(i) - 1]
    if '+' in s:
      s = s[1:]
    hits.append(s)
  else:
    hits.append('0')

# очки на блоке
html_wblocks = soup.find_all('td', {'class': 'stat-table-7', 'data-field': 'total_block'})

wblocks = []
for i in html_wblocks[1:]:
  new_soup = BeautifulSoup(str(i), 'lxml')
  s = new_soup.find_all('span', {'class': 'ts-table__active-cell'})
  if s != []:
    s = s[0].text
    if '%' in s:
      s = s[:len(i) - 1]
    if '+' in s:
      s = s[1:]
    wblocks.append(s)
  else:
    wblocks.append('0')


print(wnames)
print(wserves)
print(wdigs)
print(whits)
print(wblocks)

['Каракурт Эбрар', 'Бирюкова Ольга', 'Гончарова Наталия', 'Гарелик Анастасия', 'Боговская Мария', 'Чернова Анастасия', 'Мартинес Брайелин', 'Воронкова Ирина', 'Богданова Алина', 'Касаткина Нина', 'Руссу Виктория', 'Коржова Владислава', 'Распутина Валерианна', 'Фитисова Елизавета', 'Боброва Виктория', 'Симонова Карина', 'Геращенкова Екатерина', 'Карполь Елизавета', 'Мазина Елизавета', 'Смирнова Надежда', 'Тица Бранка', 'Гатина Светлана', 'Селиверстова Наталья', 'Лазарева Ангелина', 'Романова Валерия', 'Гатина Екатерина', 'Халецкая Мария', 'Пушина Виктория', 'Воробьева Мария', 'Столбова Юлия', 'Храбрая Юлия', 'Мамедова Александра', 'Костючик Вера', 'Лебедкина Ксения', 'Симоненко Ярослава', 'Ковалева Полина', 'Мельникова Анна', 'Жаброва Анастасия', 'Сергеева Варвара', 'Столяр Надежда', 'Ваганова Ольга', 'Сокольчик Екатерина', 'Беликова Полина', 'Емелина Ангелина', 'Павлова Елизавета', 'Антонова Вера', 'Лукьянова Елизавета', 'Шевчук Валерия', 'Гонсалес Гайла', 'Синицкая Юлия', 'Протопопова

Поиск топ клубов (женщины):

In [22]:
url = 'https://volley.ru/calendar/01J1YDG9TCC2MJ1RN5WM6H04C6/statistics'
response = requests.get(url)
encoding = chardet.detect(response.content)['encoding']
html = response.content.decode(encoding)
soup = BeautifulSoup(html, 'lxml')

tables = soup.find_all('table')
wssylki = []

whrefs = tables[0].find_all('a')
for whre in whrefs:
 whre = str(whre) # вытащили список ссылок на страницы игроков
 windex = whre.find('>')
 #print('https://volley.ru' + str(hre[9:44])) # сделали их корректными ссылками
 wssylki.append('https://volley.ru' + str(whre[9:44]))

wclubs = []
for i in range(60):
  wssylka = wssylki[i]
  response = requests.get(wssylka)
  encoding = chardet.detect(response.content)['encoding']
  html = response.content.decode(encoding)
  soup = BeautifulSoup(html, 'lxml')
  html_urls = soup.find_all('a', {'class': 'itext-40'})
  wclubs.append(html_urls[0].text)

In [23]:
wclubs_dict = {}
for item in wclubs:
    if item in wclubs_dict:
        wclubs_dict[item] += 1
    else:
        wclubs_dict[item] = 1
print(wclubs_dict)

{' Локомотив, Калининградская обл.': 3, ' Заречье-Одинцово, Московская обл.': 5, '  Динамо-Москва': 5, ' Ленинградка, Санкт-Петербург': 4, ' Динамо, Краснодар': 5, ' Динамо-Ак Барс, Казань': 5, ' Минчанка': 4, ' Динамо-Метар, Челябинск': 4, ' Омичка, Омская обл.': 3, ' Енисей, Красноярск': 4, ' Уралочка-НТМК, Свердловская обл.': 6, ' Тулица, Тульская обл.': 5, ' Спарта, Нижний Новгород': 4, '  Протон, Саратов': 3}
